In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import random
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
# flatmaps_per_question = __import__('06_flatmaps_per_question')
import viz
import gemv

Note, this notebook requires first running `03_export_qa_flatmaps.ipynb` into `df_qa_dict.pkl` files for each subject.

### load gemv average flatmaps

In [ ]:
gemv_avgs_pilot = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot.pkl'))
gemv_avgs_pilot5 = joblib.load(join(
    RESULTS_DIR, "processed", "flatmaps", 'resps_avg_dict_pilot5.pkl'))
gemv_avg_flatmaps = gemv_avgs_pilot | gemv_avgs_pilot5
qa_list, gemv_list = gemv.get_matched_lists()

In [ ]:
# # check that gpt4 was run for all the questions
# subject = 'UTS02'
# setting = 'individual_gpt4'
# flatmaps_qa = joblib.load(
#     join(PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
# for k in qa_list:
#     assert k in flatmaps_qa.keys()

### make comparisons

In [ ]:
# load qa weights
subject = 'UTS02'
corrs_df_dict = {}
frac_voxels_to_keep_list = [0.01, 0.05, 0.1, 0.25, 0.5, 1]
for j, frac_voxels_to_keep in enumerate(frac_voxels_to_keep_list):
    flatmaps_qa_list = defaultdict(list)
    # , 'individual_35', full_35', 'shapley_35', 'individual_gpt4',
    # for setting in ['individual_gpt4', 'shapley_35']:
    # for setting in ['individual_gpt4', 'individual_gpt4_wordrate']:
    for setting in ['individual_gpt4']:  # , 'shapley_35']:
        # for setting in ['full_35_wordrate']:
        flatmaps_qa = joblib.load(
            join(PROCESSED_DIR, subject.replace('UT', ''), setting + '.pkl'))
        for q in flatmaps_qa.keys():
            flatmaps_qa_list[q].append(flatmaps_qa[q])
    # print('lens', [len(flatmaps_qa_list[q]) for q in flatmaps_qa_list.keys()])
    flatmaps_qa = {
        q: np.mean(flatmaps_qa_list[q], axis=0)
        for q in flatmaps_qa_list.keys()}

    # select what to plot
    # df_qa_dict = joblib.load(f'df_qa_dict_{subject.replace("UT", "")}.pkl')
    questions_names_df = pd.DataFrame({
        'qa_weight': qa_list,
        'gemv_avg_resp': gemv_list,
    })
    # filter only questions that exist in the flatmaps
    questions_names_df = questions_names_df[questions_names_df['qa_weight'].isin(
        flatmaps_qa.keys())]

    if isinstance(flatmaps_qa, pd.DataFrame):
        questions_names_df = questions_names_df[questions_names_df['qa_weight'].isin(
            flatmaps_qa.index)]
        flatmaps_qa = flatmaps_qa.loc[questions_names_df['qa_weight'].values]['weights'].values
    else:
        flatmaps_qa = [flatmaps_qa[q]
                       for q in questions_names_df['qa_weight'].values]
    flatmaps_gemv = [
        gemv_avg_flatmaps[q]
        for q in questions_names_df['gemv_avg_resp'].values]
    titles_gt = questions_names_df['gemv_avg_resp'].apply(
        lambda x: x[0]).astype(str)

    # mask flatmaps by corr
    # corrs_test = joblib.load(join(PROCESSED_DIR, subject.replace(
    #     'UT', ''), 'corrs_test_35.pkl')).values[0]
    # corrs_test_mask = (corrs_test >= np.percentile(
    #     corrs_test, 100 * (1 - frac_voxels_to_keep))).astype(bool)

    # flatmaps_qa = [flatmaps_qa[i][corrs_test_mask]
    #                for i in range(len(flatmaps_qa))]
    # flatmaps_gemv = [flatmaps_gemv[i][corrs_test_mask]
    #                  for i in range(len(flatmaps_gemv))]

    # mask flatmaps by extreme vals
    for i in range(len(flatmaps_qa)):
        mask_extreme = np.abs(flatmaps_qa[i]) >= np.percentile(
            np.abs(flatmaps_qa[i]), 100 * (1 - frac_voxels_to_keep))
        flatmaps_qa[i] = flatmaps_qa[i][mask_extreme]
        flatmaps_gemv[i] = flatmaps_gemv[i][mask_extreme]

    # # save flatmaps
    # if frac_voxels_to_keep in [0.1, 1]:

    #     # apply mask as nans
    #     for i in range(len(flatmaps_qa)):
    #         flatmaps_qa[i][~corrs_test_mask] = np.nan
    #         flatmaps_gemv[i][~corrs_test_mask] = np.nan

    #     for i in tqdm(range(len(flatmaps_qa))):
    #         sasc.viz.quickshow(
    #             flatmaps_qa[i],
    #             subject=subject,
    #             fname_save=join(repo_dir, 'qa_results', 'gemv', subject,
    #                             f'frac_voxels={frac_voxels_to_keep}',
    #                             setting, f'{titles_gt[i]}.png'),
    #             cmap='RdYlBu_r',
    #         )

    #         sasc.viz.quickshow(
    #             flatmaps_gemv[i],
    #             subject=subject,
    #             fname_save=join(repo_dir, 'qa_results', 'gemv', subject,
    #                             f'frac_voxels={frac_voxels_to_keep}',
    #                             'gemv', f'{titles_gt[i]}.png'),
    #             cmap='RdYlBu_r',
    #         )

    # shuffle gemv
    # random.shuffle(flatmaps_gemv)

    # print num nas
    # print('nas', [np.isnan(flatmaps_gemv[i]).sum()
    #   for i in range(len(flatmaps_gemv))])

    corrs = viz._calc_corrs(
        flatmaps_qa,
        flatmaps_gemv,
        titles_qa=[analyze_helper.abbrev_question(q)
                   for q in questions_names_df['qa_weight'].astype(str)],
        titles_gt=titles_gt,
    )
    # corrs_df = pd.DataFrame({'corrs': np.diag(
    # corrs.values), 'questions': corrs.columns}).sort_values('corrs', ascending=False)
    # corrs_df_dict[frac_voxels_to_keep] = corrs_df.copy()
    corrs_df_dict['questions'] = corrs.columns
    corrs_df_dict[f'corrs_{frac_voxels_to_keep}'] = np.diag(corrs.values)

    # corrs_df.to_pickle(join(repo_dir, 'qa_results', 'gemv',
    # setting + '_corrs_df.pkl'))

    colors = sns.color_palette('viridis', len(frac_voxels_to_keep_list))
    viz.corr_bars(
        corrs,
        out_dir_save=join(repo_dir, 'qa_results', 'gemv',
                          'corrs_' + setting),
        xlab='GEM-V',
        color=colors[j],
        label=f'{frac_voxels_to_keep * 100:.0f}%',
    )
    # plt.savefig(join(out_dir_save, 'corrs_barplot.pdf'), bbox_inches='tight')
    # plt.savefig(join(out_dir_save, 'corrs_barplot.png'),
    #             bbox_inches='tight', dpi=300)
    # plt.show()

# center legend text
plt.legend(title='Fraction of\nbest-predicted voxels\ncompared',
           labelcolor='linecolor')
# plt.tight_layout()
plt.savefig(join(repo_dir, 'qa_results', 'gemv', subject, 'corrs_barplot.pdf'),
            bbox_inches='tight')
plt.savefig(join(repo_dir, 'qa_results', 'gemv', subject, 'corrs_barplot.png'),
            bbox_inches='tight', dpi=300)
plt.show()

In [101]:
corrs_df = pd.DataFrame(corrs_df_dict)

In [102]:
flatmaps_gemv_masked = np.array([
    flatmaps_gemv[i][np.abs(flatmaps_qa[i]) >= np.percentile(
        np.abs(flatmaps_qa[i]), 100 * (1 - frac_voxels_to_keep))]
    for i in range(len(flatmaps_qa))
])

In [103]:
eng1000 = joblib.load(
    join(PROCESSED_DIR, subject.replace('UT', ''), 'eng1000_weights.pkl'))
masks_eng1000 = [
    np.abs(eng1000[i_]) >= np.percentile(
        np.abs(eng1000[i_]), 100 * (1 - frac_voxels_to_keep))
    for i_ in range(eng1000.shape[0])
]

In [110]:
for frac_voxels_to_keep in tqdm(frac_voxels_to_keep_list):
    pvals = []
    for i in range(len(flatmaps_qa)):
        corrs_perm_eng1000 = np.array(flatmaps_gemv)[
            :, masks_eng1000[i]] @ eng1000[:, masks_eng1000[i]].T
        pvals.append((corrs_perm_eng1000 >
                     corrs_df[f'corrs_{frac_voxels_to_keep}'].values[i]).mean())

    # get what fraction of 'corrs_perm_eng1000' column is greater than 'corrs'
    corrs_df[f'pval_{frac_voxels_to_keep}'] = pvals

In [109]:
# format scientific notation
corrs_df.sort_values(
    by='pval_0.01').style.background_gradient().format(precision=3)

,questions,corrs_0.01,corrs_0.05,corrs_0.1,corrs_0.25,corrs_0.5,corrs_1,pval_0.01,pval_0.05,pval_0.1,pval_0.25,pval_0.5,pval_1
2,locations,0.707,0.643,0.569,0.448,0.362,0.285,0.027,0.036,0.051,0.086,0.128,0.182
5,cultural references,0.603,0.465,0.400,0.299,0.234,0.178,0.044,0.079,0.109,0.171,0.224,0.280
16,proper nouns,0.577,0.416,0.333,0.250,0.193,0.147,0.049,0.100,0.146,0.211,0.263,0.313
20,technical or specialized terminology,0.563,0.346,0.224,0.112,0.074,0.056,0.052,0.138,0.233,0.355,0.402,0.426
6,dialogue,0.560,0.639,0.582,0.470,0.381,0.296,0.052,0.037,0.048,0.078,0.118,0.174
17,relationships between people,0.552,0.451,0.384,0.292,0.228,0.176,0.054,0.085,0.117,0.177,0.230,0.281
15,planning or organizing,0.501,0.430,0.375,0.283,0.214,0.157,0.068,0.093,0.121,0.184,0.242,0.303
10,opinions or judgments,0.499,0.368,0.304,0.212,0.156,0.113,0.068,0.125,0.167,0.244,0.304,0.355
11,personal or interactions interactions,0.471,0.343,0.281,0.232,0.192,0.151,0.077,0.140,0.185,0.225,0.264,0.308
3,communication,0.389,0.506,0.472,0.382,0.308,0.245,0.114,0.066,0.077,0.118,0.164,0.215


# Look at merged flatmaps

In [ ]:
corrs_df = pd.read_pickle(join(repo_dir, 'qa_results', 'gemv',
                               setting + '_corrs_df.pkl'))

In [ ]:
img_parent_dir = join(repo_dir, 'qa_results', 'gemv',
                      subject, 'frac_voxels=0.1')
img_dir1 = join(img_parent_dir, 'gemv')
img_dir2 = join(img_parent_dir, setting)

# read images and combine them with their filenames on a single plot
# fnames = os.listdir(img_dir1)
# fnames = [f for f in fnames if f.endswith('.png')]
# only keep the ones that are in both directories
# fnames = [f for f in fnames if f in os.listdir(img_dir2)]


# corrs = corrs.sort_values('corrs', ascending=False)
fnames = [v + '.png' for v in corrs_df['questions'].values]

n = len(fnames)
C = 4
R = int(np.ceil(n / C))

fig, axs = plt.subplots(R, C, figsize=(C * 3.2, R * 1))
axs = axs.flatten()
for i in range(len(axs)):
    axs[i].axis('off')
for i, fname in enumerate(fnames):
    img1 = plt.imread(join(img_dir1, fname))
    img2 = plt.imread(join(img_dir2, fname))
    axs[i].imshow(np.concatenate([img1, img2], axis=1))
    axs[i].set_title(
        f'{fname[:-4]} ({corrs_df["corrs"].values[i]:0.3f})', fontsize=8)

# add text in bottom right of figure
fig.text(0.99, 0.01, f'{subject}\nGEMV on left, QA on right',
         ha='right', va='bottom', fontsize=8)
plt.tight_layout()
plt.savefig(join(img_parent_dir, f'flatmaps_{setting}_{subject}.png'), dpi=300)
plt.close()